# TFRecord
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- Tensorflow에서 제공하는 데이터셋을 파일에 저장방식.
    - 데이터 양이 많을 경우 이를 Binary로 Seralization(직렬화)하여 하나의 파일로 저장하고 있다가, 이를 다시 읽어 들여  처리하면 처리속도를 향상시킬 수 있다. Tensorflow에서 이를 위해서 데이터 셋을 Protocol Buffer 형태로 Serialization을 수행해서 저장할 수 있는 TFRecord 파일 포맷 형태를 지원한다. 
    - tf.train.Example 클래스를 이용해서 {“string” : tf.train.Feature} 의 딕셔너리 형태로 데이터들을 TFRecord 파일에 저장한다.
    
> - **직렬화(Serialization):** 메모리에 저장된 다양한 타입의 값을 디스크(네트워크)에 저장할 수 있는 상태로 변환하는 것.
> - **binary data:** 디스크에 저장되는 0, 1로 구성된 데이터

- tf.train.Example
    - 하나의 데이터를 TFRecord에 저장하기 위해 변환하는 클래스. 하나의 데이터를 tf.train.Example 의 객체로 변환해서 저장한다.

- tf.train.Feature
    - 하나의 데이터를 구성하는 속성(feature)들을 변환하는 클래스.
    - tf.train.Feature는 다음 세가지 타입을 지원한다.
        - tf.train.BytesList – string, byte 타입을 변환
        - tf.train.FloatList –  float(float32), double(float64) 타입을 변환
        - tf.train.Int64List – bool, enum, int32, uint32, int64, uint64 타입을 변환

- tf.tran.Example의 형태
```python
{
    "feature명":tf.train.Feature타입객체,
    "feature명":tf.train.Feature타입객체,
    ...
}
```


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float/double datatype value"""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns a int64_list from a int / uint/ bool / enum datatype value"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
# 정수
v = _int64_feature(20)
print(type(v))
v

<class 'tensorflow.core.example.feature_pb2.Feature'>


int64_list {
  value: 20
}

In [4]:
# bool
_int64_feature(True)

int64_list {
  value: 1
}

In [5]:
_int64_feature(False)

int64_list {
  value: 0
}

In [6]:
# ndarray
_int64_feature(np.array(50).astype(np.uint32))

int64_list {
  value: 50
}

In [7]:
# float
_float_feature(30.2)

float_list {
  value: 30.200000762939453
}

In [8]:
# 문자열
_bytes_feature(b'test')

bytes_list {
  value: "test"
}

In [9]:
_bytes_feature('test'.encode('utf-8'))

bytes_list {
  value: "test"
}

In [12]:
# 파일(binary)
with open('simple_images/bear/bear_1.jpeg', 'rb') as f:
    img = f.read()
print(type(img))
_bytes_feature(img)

<class 'bytes'>


bytes_list {
  value: "\377\330\377\342\002@ICC_PROFILE\000\001\001\000\000\0020ADBE\002\020\000\000mntrRGB XYZ \007\317\000\006\000\003\000\000\000\000\000\000acspAPPL\000\000\000\000none\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\366\326\000\001\000\000\000\000\323-ADBE\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\ncprt\000\000\000\374\000\000\0002desc\000\000\0010\000\000\000kwtpt\000\000\001\234\000\000\000\024bkpt\000\000\001\260\000\000\000\024rTRC\000\000\001\304\000\000\000\016gTRC\000\000\001\324\000\000\000\016bTRC\000\000\001\344\000\000\000\016rXYZ\000\000\001\364\000\000\000\024gXYZ\000\000\002\010\000\000\000\024bXYZ\000\000\002\034\000\000\000\024text\000\000\000\000Copyright 1999 Adobe Systems Incorporated\000\000\000desc\000\000\000\000\000\000\000\021Adobe RGB (1998)\000\000\000\000\000\000\000\000\

### Feature 직렬화
- .SerializeToString()
    - proto 메세지를 bytes(binary string)로 직렬화
    - Example을 tfrecord로 출력하기 전에 변환해야 한다.

In [13]:
feature = _float_feature(30.2)
print(feature)
feature.SerializeToString()

float_list {
  value: 30.200000762939453
}



b'\x12\x06\n\x04\x9a\x99\xf1A'

### 직렬화 예제

In [14]:
# 4개 속성
# 1속성: Bool, 2: 정수, 3: 문자열, 4: 실수
import numpy as np
N_DATA = 1000 # 데이터 개수

feature0 = np.random.choice([False, True], N_DATA)

feature1 = np.random.randint(0, 5, N_DATA) # [0, 4, 1, 2...]
strings = np.array([b'cat',b'dog',b'chicken',b'horse',b'goat'])
feature2 = strings[feature1]
feature3 = np.random.randn(N_DATA)

In [16]:
feature0.dtype, feature1.dtype, feature2.dtype, feature3.dtype

((1000,), dtype('int32'), dtype('S7'), dtype('float64'))

# TFRecord 저장 및 읽기

## tf.train.Example 생성 및 직렬화(Serialize)
1. 각 관측값의 Feature들 하나하나는 위의 함수 중 하나를 사용하여 3 가지 호환 유형 중 하나를 포함하는 tf.train.Feature 로 변환(인코딩)되어야 한다.
2. Feature이름 문자열에 1번에서 에서 생성 된 인코딩 된 기능 값으로 딕셔너리를 생성한다.
3. 2 단계에서 생성 된 맵은 Features 메시지 로 변환한다.

In [17]:
import tensorflow as tf

def serialize_example(feature0, feature1, feature2, feature3):
    """
    한개의 데이터 포인트를 받아서 Serialize된 메세지(최종저장형태)를 만들어 반환
    각 속성값들을 타입에 맞는(bool, 정수, 문자열, 실수) Feature로 변환한 뒤 dictionary로 구성
    tf.train.Example 생성
    serialize 된 bytes 생성 후 반환"""
    
    feature = {
        'feature0':_int64_feature(feature0),
        'feature1':_int64_feature(feature1),
        'feature2':_bytes_feature(feature2),
        'feature3':_float_feature(feature3)
    }
    # feature dictionary를 넣어서 example을 생성
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    # Example 객체를 파일로 저장할 수 있도록 직렬화 해준 뒤 반환
    return example_proto.SerializeToString()

## 출력(저장) 처리

- \_bytes_feature() , \_float_feature() , \_int64_feature() 중 하나를 사용하여 tf.train.Feature로 각각의 값을 변환한 뒤 tf.train.Example 메시지를 만든다.
- serializeToString()을 이용해 binary string 으로 변환한다.
- tf.io.TFRecordWriter를 이용해 출력한다.

In [18]:
import os
# tfrecord 파일을 저장할 디렉토리 생성
if not os.path.isdir('tfrecord'):
    os.mkdir('tfrecord')

In [19]:
# tfrecord 파일경로
# tfrecord 파일 확장자: xxx.tfrecord, xxx.tfr, xxx.record
tfrecord_dir = "./tfrecord/data.tfrecord" 

# TFRecordWriter: tfrecord파일에 데이터를 출력하는 객체
tfrecord_writer = tf.io.TFRecordWriter(tfrecord_dir)

In [21]:
for data in zip(feature0, feature1, feature2, feature3):
    print((data[0].dtype, data[1].dtype, data[2].dtype, data[3].dtype))
    # data[0].dtype: numpy bool => numpy의 bool타입은 파이썬의 bool타입으로 변환 처리
    pb = serialize_example(bool(data[0]), data[1], data[2], data[3])
    tfrecord_writer.write(pb) # tfrecord파일에 Example 출력
tfrecord_writer.close()    

(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('flo

(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('flo

(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('flo

(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('flo

(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S5'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S4'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S7'), dtype('float64'))
(dtype('bool'), dtype('int32'), dtype('S3'), dtype('flo

In [23]:
serialized_example = serialize_example(False, 4, b'goat', 0.92)
serialized_example

b'\nR\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\x1f\x85k?'

## TFRecord파일 읽기 및 역직렬화(Deserialize)
- tfrecord 파일로 저장된 직렬화된 데이터를 읽어 들어와서 feature들을 parsing
- tf.data.TFRecordDataset를 이용해 읽는다.

In [26]:
def _parse_function(tfrecord_serialized):
    """매개변수로 직렬화된 Example(1개)을 받아서 feature0~4 추출해서 반환"""
    # Feature들을 지정할 Dictionary 생성
    # 저장할 때 지정한 이름: FixedLenFeature() # 저장된 값을 읽어서 저장할 빈 Feature에 저장
    features = {
        "feature0":tf.io.FixedLenFeature([], tf.int64),
        "feature1":tf.io.FixedLenFeature([], tf.int64),
        "feature2":tf.io.FixedLenFeature([], tf.string),
        "feature3":tf.io.FixedLenFeature([], tf.float32)
    }
    # 직렬화된 Example과 빈 Feature 딕셔너리를 넣어서 Example 객체 생성
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features) 
    
    # Example에서 Feature들을 추출
    # 추출된 값은 Feature 타입이므로 원래 타입으로 형변환을 진행
    feature0 = tf.cast(parsed_features['feature0'], tf.bool) # Int64List -> bool
    feature1 = tf.cast(parsed_features['feature1'], tf.int64) # Int64List -> int64
    feature2 = tf.cast(parsed_features['feature2'], tf.string) # Int64List -> string
    feature3 = tf.cast(parsed_features['feature3'], tf.float32) # Int64List -> float
    
    return feature0, feature1,feature2,feature3

In [25]:
# TFRecordDataset(tfrecord 파일경로) 이용해서 input 작업 처리
# 변환작업: map() 이용
dataset = tf.data.TFRecordDataset(tfrecord_dir)
type(dataset)

tensorflow.python.data.ops.readers.TFRecordDatasetV2

In [27]:
for raw_data in dataset.take(3):
    print(repr(raw_data))

<tf.Tensor: shape=(), dtype=string, numpy=b'\nR\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\x81G\xa8?\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nQ\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x01\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04}\xe3\xe6\xbf\n\x13\n\x08feature2\x12\x07\n\x05\n\x03dog\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nQ\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x01\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\x8a)c?\n\x13\n\x08feature2\x12\x07\n\x05\n\x03dog\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00'>


In [28]:
parsed_dataset = dataset.map(_parse_function)

In [29]:
type(parsed_dataset)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [33]:
for raw in parsed_dataset.take(10):  #(f1, f2, f3, f4) 튜플로 반환됨.
#     print(raw)
    print(raw[0].numpy(), raw[1].numpy(), raw[2].numpy(), raw[3].numpy())

False 4 b'goat' 1.3146821
False 1 b'dog' -1.8038174
False 1 b'dog' 0.8873526
True 1 b'dog' -0.6239882
False 3 b'horse' 1.6122534
True 2 b'chicken' 1.053527
False 0 b'cat' -0.77071977
True 4 b'goat' 0.4419014
True 0 b'cat' 1.787691
True 0 b'cat' 1.1507446
